In [1]:
group_key = ''

In [2]:
import vk_api 
from vk_api.longpoll import VkLongPoll, VkEventType
from vk_api.utils import get_random_id
from vk_api.keyboard import VkKeyboard, VkKeyboardColor
import pandas as pd
import numpy as np
import logging

## Logging

In [3]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s,%(levelname)s,%(message)s')

file_handler = logging.FileHandler('billiard.log')
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(formatter)

debug_file_handler = logging.FileHandler('debug_billiard.log', mode = 'w')
debug_file_handler.setLevel(logging.DEBUG)
debug_file_handler.setFormatter(formatter)

logger.addHandler(file_handler)
logger.addHandler(debug_file_handler)

## Dictionary

In [4]:
CONTEXT_CHANGE_INFO = 'Введи данные, которые хочешь изменить в формате: \n ФИО: Иванов Иван Иванович \n или \n Курс: 1'
CONTEXT_CHOOSE_ACTION = 'Ты можешь проверить свои знания, выбрав конкретную тему, или посмотреть результаты в разделе "Статистика"'
CONTEXT_CHOOSE_TOPIC = 'Выбери тему, по которой хочешь пройти тестирование'
CONTEXT_GREET_USER = 'Привет! С тобой на связи бильярдный клуб МИСИС. Прежде чем приступить к тестам, давай познакомимся. Введи ниже свое ФИО в формате: \n ФИО: Иванов Иван Иванович'
CONTEXT_ENTER_COURSE = 'Теперь введи номер своего курса в формате: \n Курс: 1'
CONTEXT_LAST_QUESTION = 'На этом все! Ты можешь пройти тесты еще раз или посмотреть статистику'
CONTEXT_LETS_START = 'Отлично! Теперь можешь приступать к тестированию'
CONTEXT_MENU = 'Выберите раздел:'
CONTEXT_OTHER = 'Этот раздел находится на этапе разработки. Мы пришлем вам уведомление, когда здесь появятся новые возможности'
CONTEXT_RIGHT = 'Верно!'
CONTEXT_SUCCESSFUL = 'Из них правильных ответов:'
CONTEXT_TOTAL = 'Всего ответов:'
CONTEXT_USER_NAME = 'фио:'
CONTEXT_COURSE = 'курс:'
CONTEXT_WRONG = 'Неверно. '
TITLE_BACK_TO_MENU = 'Назад в меню'
TITLE_CHANGE_INFO = 'Редактировать личные данные'
TITLE_CHOOSE_TOPIC = 'Выбрать тему'
TITLE_FOULS = 'Фолы'
TITLE_NEXT = 'Дальше'
TITLE_OTHER = 'Другое'
TITLE_START = 'Начать'
TITLE_STATS = 'Статистика'
TITLE_STOP = 'Завершить'
TITLE_TERMS = 'Терминология бильярда'
TITLE_TESTS = 'Тесты'

## Keyboards

In [5]:
menu = VkKeyboard()
menu.add_button(TITLE_TESTS, color=VkKeyboardColor.PRIMARY)
menu.add_button(TITLE_CHANGE_INFO, color=VkKeyboardColor.PRIMARY)
menu.add_button(TITLE_OTHER, color=VkKeyboardColor.SECONDARY)

In [6]:
test_keyboard = VkKeyboard()
test_keyboard.add_button(TITLE_CHOOSE_TOPIC, color=VkKeyboardColor.PRIMARY)
test_keyboard.add_button(TITLE_STATS, color=VkKeyboardColor.PRIMARY)
test_keyboard.add_button(TITLE_BACK_TO_MENU, color=VkKeyboardColor.SECONDARY)

In [7]:
theme_keyboard = VkKeyboard()
theme_keyboard.add_button(TITLE_TERMS, color=VkKeyboardColor.PRIMARY)
theme_keyboard.add_button(TITLE_FOULS, color=VkKeyboardColor.PRIMARY)
theme_keyboard.add_button(TITLE_BACK_TO_MENU, color=VkKeyboardColor.SECONDARY)

In [8]:
answer_keyboard_4 = VkKeyboard()
answer_keyboard_4.add_button(1, color=VkKeyboardColor.SECONDARY)
answer_keyboard_4.add_button(2, color=VkKeyboardColor.SECONDARY)
answer_keyboard_4.add_button(3, color=VkKeyboardColor.SECONDARY)
answer_keyboard_4.add_button(4, color=VkKeyboardColor.SECONDARY)

In [9]:
answer_keyboard_2 = VkKeyboard()
answer_keyboard_2.add_button(1, color=VkKeyboardColor.SECONDARY)
answer_keyboard_2.add_button(2, color=VkKeyboardColor.SECONDARY)

In [10]:
action_choose = VkKeyboard(one_time=True)
action_choose.add_button(TITLE_NEXT, color=VkKeyboardColor.SECONDARY)
action_choose.add_button(TITLE_STOP, color=VkKeyboardColor.SECONDARY)

## Globals

In [11]:
is_testing = False
right_answer = ''
indx = 0
current_test = pd.DataFrame()
answer_keyboard = VkKeyboard()

In [12]:
df = pd.read_excel('results.xlsx')
tests_term = pd.read_excel('billiard_tests_term.xlsx')
test_fouls = pd.read_excel('tests_fouls.xlsx')

## Functions

### Utils

In [13]:
def get_user_name(text):
    return text[4:].strip()

In [14]:
def get_course(text):
    return text[5:].strip()

In [15]:
def is_user_answer(text):
    if text == '1' or text == '2' or text == '3' or text == '4':
        return True
    return False

### Common

In [16]:
def greeting(event, vk):
    vk.messages.send(user_id = event.user_id,
                     random_id = get_random_id(),
                     message = CONTEXT_GREET_USER)

In [17]:
def enter_course(event, vk):
    vk.messages.send(user_id = event.user_id,
                     random_id = get_random_id(),
                     message = CONTEXT_ENTER_COURSE)

In [18]:
def first_start(event, vk):
    vk.messages.send(user_id = event.user_id,
                    random_id = get_random_id(),
                    message = CONTEXT_LETS_START,
                    keyboard = menu.get_keyboard())

In [19]:
def start(event, vk):
    vk.messages.send(user_id = event.user_id,
                    random_id = get_random_id(),
                    message=CONTEXT_MENU,
                    keyboard = menu.get_keyboard())

In [20]:
def send_test_menu(event, vk):
    vk.messages.send(user_id = event.user_id,
                    random_id = get_random_id(),
                    message = CONTEXT_CHOOSE_ACTION,
                    keyboard = test_keyboard.get_keyboard())

In [21]:
def choose_topic(event, vk):
    vk.messages.send(user_id = event.user_id,
                    random_id = get_random_id(),
                    message = CONTEXT_CHOOSE_TOPIC,
                    keyboard = theme_keyboard.get_keyboard())

In [22]:
def change_info(event, vk):
    vk.messages.send(user_id = event.user_id,
                    random_id = get_random_id(),
                    message = CONTEXT_CHANGE_INFO)

In [23]:
def other(event, vk):
    vk.messages.send(user_id = event.user_id,
                    random_id = get_random_id(),
                    message = CONTEXT_OTHER)

### Test

In [24]:
def generate_question(vk, event):
    global right_answer
    question = current_test['Вопрос'].loc[indx]
    right_answer = str(current_test['Верный ответ'].loc[indx])
    video_id = ''
    if ('video_id' in current_test.columns): 
        video_id = str(current_test['video_id'].loc[indx])
    vk.messages.send(user_id = event.user_id,
                     random_id = get_random_id(),
                     message = question,
                     keyboard = answer_keyboard.get_keyboard(),
                     attachment = f'video-216098865_{video_id}')
    

In [25]:
def test_step(event, vk):
        global is_testing, indx
        is_testing = True
        generate_question(vk, event)
        indx = indx + 1

In [26]:
def is_last_question(vk, event):
        global indx
        indx = 0
        df.to_excel('results.xlsx', index=False)
        vk.messages.send(user_id = event.user_id,
                        random_id = get_random_id(),
                        message = CONTEXT_LAST_QUESTION,
                        keyboard = test_keyboard.get_keyboard())

In [27]:
def answer_is_right(vk, event):
    df['right_tries'].loc[df['id'] == event.user_id] += 1
    df['total'].loc[df['id'] == event.user_id] += 1
    vk.messages.send(user_id = event.user_id,
                     random_id = get_random_id(),
                     message = CONTEXT_RIGHT,
                     keyboard = action_choose.get_keyboard())
    if indx >= len(current_test):
        logger.info(f'end is reached,test,{event.user_id}')
        is_last_question(vk, event)

In [28]:
def answer_is_wrong(vk, event):
    df['total'].loc[df['id'] == event.user_id] += 1
    comment = str(current_test['Объяснение'].loc[indx - 1])
    vk.messages.send(user_id = event.user_id,
                     random_id = get_random_id(),
                     message = CONTEXT_WRONG + comment,
                     keyboard = action_choose.get_keyboard())
    if indx >= len(current_test):
        logger.info(f'end is reached,test,{event.user_id}')
        is_last_question(vk, event)

### Statistic

In [29]:
def send_stats(vk, event):
    total = df['total'].loc[df['id'] == event.user_id].values[0]
    success = df['right_tries'].loc[df['id'] == event.user_id].values[0]
    message = f'{CONTEXT_TOTAL} {total} \n {CONTEXT_SUCCESSFUL} {success}'
    vk.messages.send(user_id = event.user_id,
                     random_id = get_random_id(),
                     message = message,
                     keyboard = menu.get_keyboard())

## Main

In [30]:
vk_session = vk_api.VkApi(token = group_key)
longpoll = VkLongPoll(vk_session)
vk = vk_session.get_api()

In [31]:

for event in longpoll.listen():
    if event.type == VkEventType.MESSAGE_NEW and event.to_me:
        #chat start
        if event.text == TITLE_START:
            logger.info(f'button start,start,{event.user_id}')
            if  event.user_id in df['id'].values:
                start(event, vk)
            else:
                greeting(event, vk)
        #user name
        if event.text.lower().startswith(CONTEXT_USER_NAME):
            if event.user_id in df['id'].values:
                df['name'].loc[df['id'] == event.user_id] = get_user_name(event.text)
                logger.info(f'user changed name,contact info,{event.user_id}')
                first_start(event, vk)
            else:
                df.loc[len(df.index)] = [event.user_id,  get_user_name(event.text), 0, 0, False]
                logger.info(f'new user,contact info,{event.user_id}')
                enter_course(event, vk)
        #course info
        if event.text.lower().startswith(CONTEXT_COURSE):
            try:
                if 1 <= int(get_course(event.text)) < 3:
                    df['PE'].loc[df['id'] == event.user_id] = True
                logger.info(f'PE flag,contact info,{event.user_id}')
                first_start(event, vk)
            except:
                logger.error(f'incorrect course number,contact info,{event.user_id}')
        #change info
        if event.text == TITLE_CHANGE_INFO:
            change_info(event, vk)
        #menu: TEST
        if event.text == TITLE_TESTS:
            send_test_menu(event, vk)
            logger.info(f'test,menu,{event.user_id}')
        #menu: OTHER
        if event.text == TITLE_OTHER:
            other(event, vk)
            logger.info(f'other,menu,{event.user_id}')
        #choose topic
        if event.text == TITLE_CHOOSE_TOPIC:
            choose_topic(event, vk)  
            logger.info(f'choose topic,menu,{event.user_id}')
        #test: TERMS
        if event.text == TITLE_TERMS:
            logger.info(f'topic terms,test,{event.user_id}')
            current_test = tests_term.sample(frac=1).reset_index(drop=True)
            answer_keyboard = answer_keyboard_4
            test_step(event, vk)
        #test: FOULS
        if event.text == TITLE_FOULS:
            logger.info(f'topic fouls,test,{event.user_id}')
            current_test = test_fouls.sample(frac=1).reset_index(drop=True)
            answer_keyboard = answer_keyboard_2
            test_step(event, vk)
        #check answer
        if is_testing:
            if is_user_answer(event.text): 
                if right_answer == event.text:
                    answer_is_right(vk, event)
                    is_testing = False
                else:
                    answer_is_wrong(vk, event)
                    is_testing = False
            elif event.text != TITLE_FOULS and event.text != TITLE_TERMS:
                logger.warning(f'incorrect answer,test,{event.user_id}')
        #next question
        if event.text == TITLE_NEXT:
            test_step(event, vk)
        #stop questions
        if event.text == TITLE_STOP:
            logger.info(f'session interrupted,test,{event.user_id}')
            is_last_question(vk, event)
        #get statistics
        if event.text == TITLE_STATS:
            logger.info(f'statistics,statistics,{event.user_id}')
            send_stats(vk, event)
        #go back to menu
        if event.text == TITLE_BACK_TO_MENU:
            logger.info(f'back to main,menu,{event.user_id}')
            start(event, vk)

            
            

C:\Users\bychk\AppData\Local\Temp\ipykernel_1792\2393728587.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['PE'].loc[df['id'] == event.user_id] = True
C:\Users\bychk\AppData\Local\Temp\ipykernel_1792\2393728587.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['name'].loc[df['id'] == event.user_id] = get_user_name(event.text)
C:\Users\bychk\AppData\Local\Temp\ipykernel_1792\1530300423.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-

ReadTimeout: HTTPSConnectionPool(host='im.vk.com', port=443): Read timed out. (read timeout=35)